In [54]:
select * from Customer

select * from Restaurant

select * from RestaurantReview



(3 rows affected)

(2 rows affected)

(1 row affected)

Total execution time: 00:00:00.005

customer_id,customer_name,customer_address,phone_number,email_address
10,razvan,dunarii1,0712121212,cosmin@example.com
11,customer1,test1,0000000000,test1@gmail.ro
12,customer1,test1,0000000000,test1@gmail.ro


restaurant_id,restaurant_name,restaurant_description,restaurant_phone_number,restaurant_address
4,restaurant2,A fine dining restaurant with a variety of gourmet dishes.,0700000000,Cluj-Napoca
5,Grande,A fine dining restaurant with a variety of gourmet dishes.,0700000000,Cluj-Napoca


restaurant_review_id,customer_id,restaurant_id,rating,comment
3,10,4,5.0,NULL


In [21]:
/*CREATE TABLE RestaurantReview(
    restaurant_review_id INT IDENTITY(1, 1) PRIMARY KEY,
    customer_id INT FOREIGN KEY REFERENCES Customer(customer_id),
    restaurant_id INT FOREIGN KEY REFERENCES Restaurant(restaurant_id),
    rating DECIMAL(2,1) check (rating >= 0 and rating <= 5) DEFAULT 5,
    comment VARCHAR(500) NOT NULL,
);

CREATE TABLE Restaurant(
    restaurant_id INT IDENTITY(1,1) PRIMARY KEY,
    restaurant_name VARCHAR(50) NOT NUll,
    restaurant_description VARCHAR(500),
    restaurant_phone_number VARCHAR(20) NOT NULL,
    restaurant_address VARCHAR(200) NOT NULL
);

CREATE TABLE Customer(
    customer_id INT IDENTITY(1, 1) PRIMARY KEY,
    customer_name VARCHAR(50) NOT NULL,
    customer_address VARCHAR(200) NOT NULL,
    phone_number VARCHAR(20) NOT NULL,
    email_address VARCHAR(50) NOT NULL
)*/



: Msg 3701, Level 11, State 5, Line 28
Cannot drop the table 'RestaurantReview', because it does not exist or you do not have permission.

Total execution time: 00:00:00.002

In [38]:
CREATE OR ALTER FUNCTION ValidateCustomer(
    @customer_name VARCHAR(50),
    @customer_address VARCHAR(200),
    @phone_number VARCHAR(20),
    @email_address VARCHAR(50)
)
RETURNS BIT
AS
BEGIN
    DECLARE @IsValid BIT
    SET @IsValid = 1

    IF @customer_name IS NULL OR LEN(@customer_name) = 0 OR LEN(@customer_name) > 50
    BEGIN
        SET @IsValid = 0
    END

    IF @customer_address IS NULL OR LEN(@customer_address) = 0 OR LEN(@customer_address) > 200
    BEGIN
        SET @IsValid = 0
    END

    IF @phone_number IS NULL OR LEN(@phone_number) = 0 OR LEN(@phone_number) > 20
    BEGIN
        SET @IsValid = 0
    END

    IF @email_address IS NULL OR LEN(@email_address) = 0 OR LEN(@email_address) > 50 OR
       @email_address NOT LIKE '%_@__%.__%'
    BEGIN
        SET @IsValid = 0
    END

    RETURN @IsValid
END
GO
CREATE OR ALTER FUNCTION ValidateRestaurant(
    @restaurant_name VARCHAR(50),
    @restaurant_description VARCHAR(500),
    @restaurant_phone_number VARCHAR(20),
    @restaurant_address VARCHAR(200)
)
RETURNS BIT
AS
BEGIN
    DECLARE @IsValid BIT
    SET @IsValid = 1

    IF @restaurant_name IS NULL OR LEN(@restaurant_name) = 0 OR LEN(@restaurant_name) > 50
    BEGIN
        SET @IsValid = 0
    END

    IF @restaurant_description IS NOT NULL AND LEN(@restaurant_description) > 500
    BEGIN
        SET @IsValid = 0
    END

    IF @restaurant_phone_number IS NULL OR LEN(@restaurant_phone_number) = 0 OR LEN(@restaurant_phone_number) > 20
    BEGIN
        SET @IsValid = 0
    END

    IF @restaurant_address IS NULL OR LEN(@restaurant_address) = 0 OR LEN(@restaurant_address) > 200
    BEGIN
        SET @IsValid = 0
    END

    RETURN @IsValid
END

GO
CREATE OR ALTER FUNCTION ValidateRestaurantReview(
    @rating DECIMAL(2,1)
)
RETURNS BIT
AS
BEGIN
    DECLARE @IsValid BIT
    SET @IsValid = 1

    IF @rating IS NOT NULL AND  (@rating < 0 OR @rating > 5)
    BEGIN
        SET @IsValid = 0
    END

    RETURN @IsValid
END


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.018

In [43]:
CREATE OR ALTER PROCEDURE AddCustomerRestaurantReview
    @customer_name VARCHAR(50),
    @customer_address VARCHAR(200),
    @phone_number VARCHAR(20),
    @email_address VARCHAR(50),
    @restaurant_name VARCHAR(50),
    @restaurant_description VARCHAR(500),
    @restaurant_phone_number VARCHAR(20),
    @restaurant_address VARCHAR(200)
AS
BEGIN

    BEGIN TRANSACTION

    BEGIN TRY
        IF dbo.ValidateCustomer(@customer_name, @customer_address, @phone_number, @email_address) = 0
        BEGIN
            RAISERROR('Invalid customer data', 14 , 1)
        END

        IF dbo.ValidateRestaurant(@restaurant_name, @restaurant_description, @restaurant_phone_number, @restaurant_address) = 0
        BEGIN
            RAISERROR('Invalid restaurant data', 14, 1)
        END


        INSERT INTO Customer (customer_name, customer_address, phone_number, email_address)
        VALUES (@customer_name, @customer_address, @phone_number, @email_address);

        DECLARE @customer_id INT
        SET @customer_id = SCOPE_IDENTITY();

        INSERT INTO Restaurant (restaurant_name, restaurant_description, restaurant_phone_number, restaurant_address)
        VALUES (@restaurant_name, @restaurant_description, @restaurant_phone_number, @restaurant_address);

        DECLARE @restaurant_id INT
        SET @restaurant_id = SCOPE_IDENTITY();

        INSERT INTO RestaurantReview (customer_id, restaurant_id)
        VALUES (@customer_id, @restaurant_id);

        COMMIT TRANSACTION;
        SELECT 'Transaction committed'
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION;
        SELECT 'Transaction rollbacked'
    END CATCH
END;


Commands completed successfully.

Total execution time: 00:00:00.009

In [7]:
select * from customer
select * from Restaurant
select * from RestaurantReview

EXEC AddCustomerRestaurantReview
    @customer_name = 'Cosmin',
    @customer_address = 'Cluj-Napoca',
    @phone_number = '0712121212',
    @email_address = 'cosmin@example.com',
    @restaurant_name = 'Grande',
    @restaurant_description = 'A fine dining restaurant with a variety of gourmet dishes.',
    @restaurant_phone_number = '0700000000',
    @restaurant_address = 'Cluj-Napoca'

select * from customer
select * from Restaurant
select * from RestaurantReview


(0 rows affected)

(0 rows affected)

(0 rows affected)

(1 row affected)

(1 row affected)

...

restaurant_review_id,customer_id,restaurant_id,rating,comment


(No column name)
Transaction committed


customer_id,customer_name,customer_address,phone_number,email_address
10,Cosmin,Cluj-Napoca,0712121212,cosmin@example.com


restaurant_id,restaurant_name,restaurant_description,restaurant_phone_number,restaurant_address
4,Grande,A fine dining restaurant with a variety of gourmet dishes.,0700000000,Cluj-Napoca


restaurant_review_id,customer_id,restaurant_id,rating,comment
3,10,4,5.0,NULL


In [8]:
select * from customer
select * from Restaurant
select * from RestaurantReview

EXEC AddCustomerRestaurantReview
    @customer_name = 'Cosmin',
    @customer_address = 'Cluj-Napoca',
    @phone_number = '0712121212',
    @email_address = '',
    @restaurant_name = 'Grande',
    @restaurant_description = 'A fine dining restaurant with a variety of gourmet dishes.',
    @restaurant_phone_number = '0700000000',
    @restaurant_address = 'Cluj-Napoca'

select * from customer
select * from Restaurant
select * from RestaurantReview

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

restaurant_review_id,customer_id,restaurant_id,rating,comment
3,10,4,5.0,NULL


(No column name)
Transaction rollbacked


customer_id,customer_name,customer_address,phone_number,email_address
10,Cosmin,Cluj-Napoca,0712121212,cosmin@example.com


restaurant_id,restaurant_name,restaurant_description,restaurant_phone_number,restaurant_address
4,Grande,A fine dining restaurant with a variety of gourmet dishes.,0700000000,Cluj-Napoca


restaurant_review_id,customer_id,restaurant_id,rating,comment
3,10,4,5.0,NULL


In [47]:
CREATE OR ALTER PROCEDURE AddCustomerRestaurantReviewV2
    @customer_name VARCHAR(50),
    @customer_address VARCHAR(200),
    @phone_number VARCHAR(20),
    @email_address VARCHAR(50),
    @restaurant_name VARCHAR(50),
    @restaurant_description VARCHAR(500),
    @restaurant_phone_number VARCHAR(20),
    @restaurant_address VARCHAR(200)
AS
BEGIN
    DECLARE @customer_id INT = -1, @restaurant_id INT = -1;

    BEGIN TRANSACTION;
    BEGIN TRY

        IF dbo.ValidateCustomer(@customer_name, @customer_address, @phone_number, @email_address) = 0
        BEGIN
            RAISERROR('Invalid customer data', 14 , 1)
        END
        INSERT INTO Customer (customer_name, customer_address, phone_number, email_address)
        VALUES (@customer_name, @customer_address, @phone_number, @email_address);
        
        SET @customer_id = SCOPE_IDENTITY();
        COMMIT TRANSACTION;
        SELECT 'Customer transaction committed';
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION;
        SELECT 'Customer transaction  rollbacked'
    END CATCH

    BEGIN TRANSACTION;
    BEGIN TRY
        IF dbo.ValidateRestaurant(@restaurant_name, @restaurant_description, @restaurant_phone_number, @restaurant_address) = 0
        BEGIN
            RAISERROR('Invalid restaurant data', 14, 1)
        END
        INSERT INTO Restaurant (restaurant_name, restaurant_description, restaurant_phone_number, restaurant_address)
        VALUES (@restaurant_name, @restaurant_description, @restaurant_phone_number, @restaurant_address);
        
        SET @restaurant_id = SCOPE_IDENTITY();
        
        COMMIT TRANSACTION;
        SELECT 'Restaurant transaction committed';
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION;
        SELECT 'Restaurant transaction rollbacked';
    END CATCH;

    BEGIN TRANSACTION;
    BEGIN TRY

        IF @customer_id = -1 OR @restaurant_id = -1
        BEGIN
          RAISERROR('Invalid ids!', 14, 1)
        END

        INSERT INTO RestaurantReview (customer_id, restaurant_id)
        VALUES (@customer_id, @restaurant_id);
        
    
        COMMIT TRANSACTION;
        SELECT 'RestaurantReview transaction committed';
    END TRY
    BEGIN CATCH
       ROLLBACK TRANSACTION;
       SELECT 'RestaurantReview transaction rollbacked';
    END CATCH;
END;


Commands completed successfully.

Total execution time: 00:00:00.033

In [9]:
select * from customer
select * from Restaurant
select * from RestaurantReview

EXEC AddCustomerRestaurantReviewV2
    @customer_name = 'Cosmin',
    @customer_address = 'Cluj-Napoca',
    @phone_number = '0712121212',
    @email_address = '',
    @restaurant_name = 'Grande',
    @restaurant_description = 'A fine dining restaurant with a variety of gourmet dishes.',
    @restaurant_phone_number = '0700000000',
    @restaurant_address = 'Cluj-Napoca'

select * from customer
select * from Restaurant
select * from RestaurantReview

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(No column name)
Restaurant transaction committed


(No column name)
RestaurantReview transaction rollbacked


customer_id,customer_name,customer_address,phone_number,email_address
10,Cosmin,Cluj-Napoca,0712121212,cosmin@example.com


restaurant_id,restaurant_name,restaurant_description,restaurant_phone_number,restaurant_address
4,Grande,A fine dining restaurant with a variety of gourmet dishes.,0700000000,Cluj-Napoca
5,Grande,A fine dining restaurant with a variety of gourmet dishes.,0700000000,Cluj-Napoca


restaurant_review_id,customer_id,restaurant_id,rating,comment
3,10,4,5.0,NULL


In [49]:
select * from customer
select * from Restaurant
select * from RestaurantReview

EXEC AddCustomerRestaurantReviewV2
    @customer_name = 'Cosmin',
    @customer_address = 'Cluj-Napoca',
    @phone_number = '0712121212',
    @email_address = 'cosmin@gmail.com',
    @restaurant_name = '',
    @restaurant_description = 'A fine dining restaurant with a variety of gourmet dishes.',
    @restaurant_phone_number = '0700000000',
    @restaurant_address = 'Cluj-Napoca'

select * from customer
select * from Restaurant
select * from RestaurantReview

(1 row affected)

(2 rows affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(No column name)
Restaurant transaction rollbacked


(No column name)
RestaurantReview transaction rollbacked


customer_id,customer_name,customer_address,phone_number,email_address
1,Cosmin,Cluj-Napoca,0712121212,cosmin@example.com
2,Cosmin,Cluj-Napoca,0712121212,cosmin@gmail.com


restaurant_id,restaurant_name,restaurant_description,restaurant_phone_number,restaurant_address
1,Grande,A fine dining restaurant with a variety of gourmet dishes.,0700000000,Cluj-Napoca
2,Grande,A fine dining restaurant with a variety of gourmet dishes.,0700000000,Cluj-Napoca


restaurant_review_id,customer_id,restaurant_id,rating,comment
1,1,1,5.0,NULL


In [58]:
-- Dirty Reads
-- T1: update + delay + rollback
-- T2: select + delay + select
BEGIN TRAN
UPDATE CUSTOMER SET customer_address='test'
where customer_id = 2
WAITFOR DELAY '00:00:10'
ROLLBACK TRAN

(1 row affected)

Total execution time: 00:00:10.006

In [ ]:
SET TRAN ISOLATION LEVEL READ UNCOMMITTED
BEGIN TRAN
select * from Customer
WAITFOR DELAY '00:00:15'
select * from Customer
COMMIT TRAN

In [63]:
select * from customer

        

(3 rows affected)

Total execution time: 00:00:00.003

customer_id,customer_name,customer_address,phone_number,email_address
1,Cosmin,Cluj-Napoca,0712121212,cosmin@example.com
2,Cosmin,Cluj-Napoca,0712121212,cosmin@gmail.com
3,alex,moisei,1234567890,test@test.ro


In [ ]:
-- Non repeatable reads
-- T1: delay + update + commit
-- T2: select + delay + select

BEGIN TRAN
WAITFOR DELAY '00:00:05'
UPDATE CUSTOMER SET customer_address = 'moisei'
where customer_name = 'alex'
COMMIT TRAN


SET TRAN ISOLATION LEVEL READ COMMITTED
BEGIN TRAN
select * from Customer
WAITFOR DELAY '00:00:05'
select * from Customer
COMMIT TRAN

-- solution ISOLATION LEVEL REPEATABLE READ


In [ ]:
-- phantom reads
-- T1: delay + insert + commit
-- T2: select + delay + select

BEGIN TRAN
WAITFOR DELAY '00:00:04'
INSERT INTO Customer (customer_name, customer_address, phone_number, email_address)
        VALUES ('customer1', 'test1', '0000000000', 'test1@gmail.ro');
COMMIT TRAN
        


SET TRAN ISOLATION LEVEL REPEATABLE READ
BEGIN TRAN
select * from Customer
WAITFOR DELAY '00:00:05'
select * from Customer
COMMIT TRAN

-- solution set level to serializable



In [69]:
select * from customer

delete from customer where customer_name = 'customer1'

(3 rows affected)

(0 rows affected)

Total execution time: 00:00:00.004

customer_id,customer_name,customer_address,phone_number,email_address
1,Cosmin,Cluj-Napoca,0712121212,cosmin@example.com
2,Cosmin,Cluj-Napoca,0712121212,cosmin@gmail.com
3,alex,moisei2,1234567890,test@test.ro


In [4]:
select * from Customer

select * from Restaurant

(3 rows affected)

Total execution time: 00:00:05.006

customer_id,customer_name,customer_address,phone_number,email_address
1,razvan,Cluj-Napoca,0712121212,cosmin@example.com


restaurant_id,restaurant_name,restaurant_description,restaurant_phone_number,restaurant_address
1,Grande,A fine dining restaurant with a variety of gourmet dishes.,0700000000,Cluj-Napoca
2,restaurant2,A fine dining restaurant with a variety of gourmet dishes.,0700000000,Cluj-Napoca
3,Grande,A fine dining restaurant with a variety of gourmet dishes.,0700000000,Cluj-Napoca


: Msg 1222, Level 16, State 51, Line 1
Lock request time out period exceeded.

In [ ]:
-- DEAD LOCK
-- T1: update table Customer + delay + update Restaurant
-- T2: update table Restaurant + delay + update Customer

begin TRAN
update Customer set customer_name='razvan' where customer_id = 1
waitfor delay '00:00:10'
update Restaurant set restaurant_name = 'restaurant2' where restaurant_id = 2
commit tran

begin TRAN
update Restaurant set restaurant_name = 'restaurant2' where restaurant_id = 2
waitfor delay '00:00:10'
update Customer set customer_name='razvan' where customer_id = 1
commit tran


--- 1 solution to execute the stmts in the same order in both of the trans or to set the deadlock priority to choose the victim we want

-- SET DEADLOCK_PRIORITY HIGH (LOW, NORMAL, HIGH / -10 to 10)

In [41]:
use master
go

ALTER DATABASE OnlineFoodDelivery
SET allow_snapshot_isolation on 

-- ALTER DATABASE OnlineFoodDelivery
-- SET read_committed_snapshot on 

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.063

In [55]:
use master
go

ALTER DATABASE OnlineFoodDelivery
SET allow_snapshot_isolation OFF

-- ALTER DATABASE OnlineFoodDelivery
-- SET read_committed_snapshot OFF


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.020

In [29]:
select * from sys.databases

select * from sys.dm_tran_active_snapshot_database_transactions

(7 rows affected)

(0 rows affected)

Total execution time: 00:00:00.031

name,database_id,source_database_id,owner_sid,create_date,compatibility_level,collation_name,user_access,user_access_desc,is_read_only,is_auto_close_on,is_auto_shrink_on,state,state_desc,is_in_standby,is_cleanly_shutdown,is_supplemental_logging_enabled,snapshot_isolation_state,snapshot_isolation_state_desc,is_read_committed_snapshot_on,recovery_model,recovery_model_desc,page_verify_option,page_verify_option_desc,is_auto_create_stats_on,is_auto_create_stats_incremental_on,is_auto_update_stats_on,is_auto_update_stats_async_on,is_ansi_null_default_on,is_ansi_nulls_on,is_ansi_padding_on,is_ansi_warnings_on,is_arithabort_on,is_concat_null_yields_null_on,is_numeric_roundabort_on,is_quoted_identifier_on,is_recursive_triggers_on,is_cursor_close_on_commit_on,is_local_cursor_default,is_fulltext_enabled,is_trustworthy_on,is_db_chaining_on,is_parameterization_forced,is_master_key_encrypted_by_server,is_query_store_on,is_published,is_subscribed,is_merge_published,is_distributor,is_sync_with_backup,service_broker_guid,is_broker_enabled,log_reuse_wait,log_reuse_wait_desc,is_date_correlation_on,is_cdc_enabled,is_encrypted,is_honor_broker_priority_on,replica_id,group_database_id,resource_pool_id,default_language_lcid,default_language_name,default_fulltext_language_lcid,default_fulltext_language_name,is_nested_triggers_on,is_transform_noise_words_on,two_digit_year_cutoff,containment,containment_desc,target_recovery_time_in_seconds,delayed_durability,delayed_durability_desc,is_memory_optimized_elevate_to_snapshot_on,is_federation_member,is_remote_data_archive_enabled,is_mixed_page_allocation_on,is_temporal_history_retention_enabled,catalog_collation_type,catalog_collation_type_desc,physical_database_name,is_result_set_caching_on,is_accelerated_database_recovery_on,is_tempdb_spill_to_remote_store,is_stale_page_detection_on,is_memory_optimized_enabled,is_data_retention_enabled,is_ledger_on,is_change_feed_enabled
master,1,NULL,0x01,2003-04-08 09:13:36.390,160,SQL_Latin1_General_CP1_CI_AS,0,MULTI_USER,0,0,0,0,ONLINE,0,0,0,1,ON,0,3,SIMPLE,2,CHECKSUM,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,00000000-0000-0000-0000-000000000000,0,0,NOTHING,0,0,0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,0,NONE,0,0,DISABLED,0,0,0,1,1,0,DATABASE_DEFAULT,master,0,0,0,0,1,0,0,0
tempdb,2,NULL,0x01,2024-05-24 11:02:25.317,160,SQL_Latin1_General_CP1_CI_AS,0,MULTI_USER,0,0,0,0,ONLINE,0,0,0,0,OFF,0,3,SIMPLE,2,CHECKSUM,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0e7fa998-68e7-4c88-8637-96d75972d644,1,0,NOTHING,0,0,0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,0,NONE,60,0,DISABLED,0,0,0,0,1,0,DATABASE_DEFAULT,tempdb,0,0,0,0,1,0,0,0
model,3,NULL,0x01,2003-04-08 09:13:36.390,160,SQL_Latin1_General_CP1_CI_AS,0,MULTI_USER,0,0,0,0,ONLINE,0,0,0,0,OFF,0,1,FULL,2,CHECKSUM,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,00000000-0000-0000-0000-000000000000,0,0,NOTHING,0,0,0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,0,NONE,60,0,DISABLED,0,0,0,1,1,0,DATABASE_DEFAULT,model,0,0,0,0,1,0,0,0
msdb,4,NULL,0x01,2023-08-23 14:45:17.367,160,SQL_Latin1_General_CP1_CI_AS,0,MULTI_USER,0,0,0,0,ONLINE,0,0,0,1,ON,0,3,SIMPLE,2,CHECKSUM,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,7e0fb0f0-9e06-4494-84ce-d056caeafa3f,1,0,NOTHING,0,0,0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,0,NONE,60,0,DISABLED,0,0,0,1,1,0,DATABASE_DEFAULT,msdb,0,0,0,0,1,0,0,0
OnlineFoodDelivery,5,NULL,0x01,2023-10-18 16:28:49.427,160,SQL_Latin1_General_CP1_CI_AS,0,MULTI_USER,0,1,0,0,ONLINE,0,0,0,0,OFF,0,1,FULL,2,CHECKSUM,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,26467532-312d-484c-815f-d4aa04a39b4c,1,0,NOTHING,0,0,0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,0,NONE,60,0,DISABLED,0,0,0,0,1,0,DATABASE_DEFAULT,OnlineFoodDelivery,0,0,0,0,1,1,0,0
Rugby2023,6,NULL,0x01,2024-01-14 17:09:18.173,160,NULL,0,MULTI_USER,0,1,0,0,ONLINE,0,1,0,0,OFF,0,1,FULL,2,CHECKSUM,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,28edf429-cba0-4346-86a7-c943f688b67a,1,0,NOTHIN

transaction_id,transaction_sequence_num,commit_sequence_num,session_id,is_snapshot,first_snapshot_sequence_num,max_version_chain_traversed,average_version_chain_traversed,elapsed_time_seconds
